#Trajectory SAPT Analysis
Proof of concept for calculating SAPT energies using psi4 over a trajectory
###Imports

In [85]:
import MDAnalysis as mda
import psi4
import matplotlib.pyplot as plt

###Defining SAPT calculator function

In [86]:
def calculate_spat(selections: list, protein: mda.Universe):
    # Writes xyz coords from trajectory
    def write_xyz(selection, universe):
        group = universe.select_atoms('resid ' + selection)
        path = 'resid' + selection +'.xyz'

        with mda.Writer(path, group.n_atoms) as coords:
            coords.write(group)
        return path

    # Gets xyz data from saved coords
    def read_xyz(xyz_path):
        with open(xyz_path, 'r') as coord_file:
            xyz_data =[]
            coord_data = coord_file.readlines()[1:]
            images = 0
            for line in coord_data:
                if line == '\n':
                    images += 1

            for line in coord_data:
                if '.' in line:
                    xyz_data.append(line)
        return xyz_data

    # Saves coords as a string
    def save_coords(coords0, coords1):
        coord_data = '0 1\n'

        for line in coords0:
            if '|' not in line:
                items = line.split()
                line = ''
                items[0] = items[0][0]
                for item in items:
                    line += (' ' + item)
                coord_data += (line + '\n')

        coord_data += '--\n'
        coord_data += '0 1\n'

        for line in coords1:
             if '|' not in line:
                items = line.split()
                line = ''
                items[0] = items[0][0]
                for item in items:
                    line += (' ' + item)
                coord_data += (line + '\n')

        coord_data += 'units angstrom'
        print(coord_data)
        return coord_data


    xyz_1 = write_xyz(selections[0], protein)
    xyz_2 = write_xyz(selections[1], protein)

    group0_coords = read_xyz(xyz_1)
    group1_coords = read_xyz(xyz_2)

    psi_in = save_coords(group0_coords, group1_coords)
    dimer = psi4.geometry(psi_in)

    psi4.set_options({'scf_type': 'df',
                      'freeze_core': 'true'})

    energy = psi4.energy('spat0/jun-cc-pvdz', molecule=dimer)
    return energy

###Creates plot from SAPT calculation

In [87]:
if __name__ == '__main__':
    # Test files
    topology = 'testfiles/testtop.psf'
    trajectory = 'testfiles/testtraj.dcd'

    test_unv = mda.Universe(topology, trajectory)
    time_list = []
    energies = []

    for ts in test_unv.trajectory:
        time_list.append(ts)
        energies.append(calculate_spat(['1', '2'], test_unv))


    plt.plot(time_list, energies, label='Energy')
    plt.xlabel('Time(ns)')
    plt.ylabel('Energy')
    plt.savefig('SAPT.png', dpi=1200)


0 1
 N 11.73604 8.50080 -10.44528
 H 12.36512 7.83994 -10.83484
 H 12.09195 9.44153 -10.72461
 H 10.83196 8.30861 -10.96393
 C 11.66462 8.39347 -8.98323
 H 12.67261 8.56049 -8.68076
 C 10.73964 9.56993 -8.59075
 H 9.63879 9.27048 -8.96795
 H 11.09461 10.51888 -9.14200
 C 10.55657 9.93424 -7.11236
 H 11.44921 10.32263 -6.69315
 H 10.27714 8.97857 -6.59725
 S 9.24967 11.10484 -6.91010
 C 10.44390 12.49235 -7.16194
 H 11.47245 12.29443 -7.66105
 H 10.56522 13.06787 -6.25163
 H 9.88309 13.17239 -7.78997
 C 11.16655 7.12900 -8.37483
 O 10.15108 6.64862 -8.75514
--
0 1
 N 11.84096 6.64227 -7.27511
 H 12.56644 7.12491 -6.93741
 C 11.41484 5.43442 -6.51348
 H 10.66005 5.00397 -7.11752
 C 12.53216 4.36391 -6.26076
 H 12.11832 3.52241 -5.66861
 H 13.30177 4.79664 -5.48926
 C 13.31617 3.75903 -7.44814
 H 12.62364 3.67989 -8.33326
 H 13.60499 2.65400 -7.19414
 C 14.60889 4.52483 -7.80496
 H 15.27049 4.50547 -6.95580
 H 14.28076 5.53066 -8.13508
 N 15.30753 3.76300 -8.87507
 H 14.75175 3.29046 -9.5

ValidationError: Inconsistent or unspecified chg/mult: sys chg: None, frag chg: [0.0, 0.0], sys mult: None, frag mult: [1, 1]